# Imports

In [3]:
# Automatically reloads newest versions of imports if functions in any python files used are changed
# %autoreload

import pandas as pd
import numpy as np

# Load datasets

In [8]:
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')

# Data clean up

In [ ]:
# Clean up the data and features

# Feature engineering

In [ ]:
# Do something

# Training the model

In [ ]:
# Do some stuff

# Make predictions

In [ ]:
# Do some more stuff

# Create submission

In [ ]:
# y_test_pred = ...

test = pd.read_csv('../data/test.csv')
# test['prediction'] = y_test_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission.csv', index=False)